In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def AIC(orden, data, numpar):
    maxi = []
    for i in range(0,orden):
        maxi.append(-2 * i/ len(data) + 2 * numpar / len(data))
    return max(maxi)

In [ ]:
def auto(n,h,funcion):
    suma = 0
    for t in range(1,n-h):
        suma = suma + funcion[t] * funcion[t + h]
    return (1/(n - h)) * suma
def PACF(orden, funcion):
    tetha = [0]
    for i in range(orden):
        sum1 = 0
        sum2 = 0
        for j in range(1,orden+1):
            sum1 = sum1 + tetha[j-1] *auto(len(funcion),orden-j,funcion)
            sum2 = sum2 + tetha[j-1] * auto(len(funcion),j,funcion)    
        tetha.append(float(auto(len(funcion),i+1,funcion) - float(sum1))/(1- float(sum2)))
    return tetha
    
def MA1(numSamples,psi):
    normales = np.random.random(numSamples)
    ar = np.zeros(len(normales))
    ar[0] = 0
    for i in range(len(normales) - 1):
        ar[i +1] = psi*normales[i-1] +  normales[i]
    return (ar)

def AR1(numSamples, phi):
    normales = np.random.randn(numSamples)
    ar = np.zeros(len(normales))
    ar[0] = 0
    for i in range(len(normales) - 1):
        ar[i] = phi*ar[i-1] +  normales[i]
    return (ar)

In [ ]:
PACF(1,MA1(10,.5))
from google.colab import files
tabla = files.upload()
plt.plot(tabla.Date,tabla.GDP);

Saving gdp.csv to gdp.csv


AttributeError: ignored

In [ ]:
fig = plt.figure(figsize=(12, 8))
AX1 = fig.add_subplot(211)
AC = sm.graphics.tsa.plot_acf(tabla['GDP'], lags=40, ax=ax1)
AX2 = fig.add_subplot(212)
PACF = sm.graphics.tsa.plot_pacf(tabla['GDP'], lags=40, ax=ax2)

In [ ]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignorar")
stepwise_fit = auto_arima(data['GDP'], start_p = 1, start_q = 1,
                          max_p = 3, max_q = 3, m = 12,
                          start_P = 0, seasonal = True,
                          d = None, D = 1, trace = True,
                          error_action ='ignore',  
                          suppress_warnings = True, 
                          stepwise = True)          
  
stepwise_fit.summary()

In [ ]:
resi = stepwise_fit.resid()
figu = sm.qqplot(resi)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from statsmodels.tsa.statespace.sarimax import SARIMAX
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from joblib import dump, load
import warnings

In [ ]:
modl = model = SARIMAX(tabla['GDP'], 
                        order = (0, 1, 1), 
                        seasonal_order =(2, 1, 1, 12))
rest = model.fit()
  
forecast = result.predict(start = len(tabla), 
                          end = (len(tabla)-1) + 3 * 12, 
                          typ = 'GDP').rename('Predictions')

print(forecast[:5])
tabla['GDP'].plot(figsize = (12, 5), legend = True)
forecast.plot(legend = True);

In [ ]:
predic = tabla.copy()
predic = predic[['Date','GDP']].rename(columns={'x': 'y'})
predic.head()

In [ ]:
steps = 36
datos_train = predic[:-steps]
datos_test  = predic[-steps:]

print(f"Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
print(f"Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")

fig, ax = plt.subplots(figsize=(9, 4))
datos_train['GDP'].plot(ax=ax, label='train')
datos_test['GDP'].plot(ax=ax, label='test')
ax.legend();

In [ ]:
def create_predictors(y):
    '''
    Create first 10 lags of a time series.
    Calculate moving average with window 20.
    '''

    lags = y[-1:-11:-1]
    mean = np.mean(y[-20:])
    predic2 = np.hstack([lags, mean])

    return predic2

In [ ]:
forecaster = ForecasterAutoregCustom(
                    regressor      = RandomForestRegressor(random_state=123),
                    fun_predictors = create_predictors,
                    window_size    = 20
             )

forecaster.fit(y=datos_train['GDP'])
forecaster

In [ ]:
steps = 36
predic3 = forecaster.predic3(steps=steps)
predic3.head(5)
fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
predic3.plot(ax=ax, label='predictions')
ax.legend();

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags = 6)

forecaster.fit(y=datos_train['GDP'])
forecaster

# Predicciones
steps = 36
predic = forecaster.predic2(steps=steps)
predic.head(5)

In [ ]:
# Gráfico
fig, ax = plt.subplots(figsize=(9, 4))
datos_train['GDP'].plot(ax=ax, label='Train')
datos_test['GDP'].plot(ax=ax, label='Test')
predic.plot(ax=ax, label='Predicciones')
ax.legend();

In [ ]:
steps = 12
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags      = 12 
             )

lags = [10, 20]

paramet = {'n_estimators': [90, 90],
              'max_depth': [3, 5, 10]}

resultados = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = datos_train['GDP'],
                        paramet         = param_grid,
                        lags          = lags_grid,
                        steps              = steps,
                        refit              = True,
                        metric             = 'mean_squared_error',
                        initial_train_size = int(len(datos_train)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False
                   )

In [ ]:

regressor = RandomForestRegressor(max_depth=3, n_estimators=90, random_state=123)
forecaster = ForecasterAutoreg(regressor = regressor,lags = 20)

forecaster.fit(y=datos_train['GDP'])


steps = 36
predic = forecaster.predict(steps=steps)
predic.head()

fig, ax = plt.subplots(figsize=(9, 4))
datos_train['GDP'].plot(ax=ax, label='train')
datos_test['GDP'].plot(ax=ax, label='test')
predic.plot(ax=ax, label='predicciones')
ax.legend();